In [ ]:
import nengo
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
from nengo.utils.filter_design import cont2discrete
import math as mt




model = nengo.Network(label="Big Basic")

with model:
    #------------------Parameters-----------------#
    # parameters of motor input
    freq = 2    # frequency limit
    rms = 0.30  # amplitude of input 
    # excitation and activation parameters
    tauNE = 0.03
    tauA  = 0.01
    tauD  = 0.05
    calciumDelHomeo = 0.1
    f2l_rel = 1     # supposing muscle is in optimal length
    f2v_rel = 1.5   # Winter&Stark from 1.2 to 1.8 for eccentric
    # Skeleton paramters
    m = 7                       # kg
    r = 0.5                     # m
    I = 0.333 * m * r**2        # kg m^2
    Mp = m*10*0.5*r*0.5           # Weight Moment around shoulder
    # simulation parameters
    seed = 0        # fixed for deterministic results
    sim_t = 10      # length of simulation
    dt = 0.001      # default nengo time step
    
    #-----------------Model-----------------------#
    #---------------------------------------------#
    # M1 comes out from MC, (PMA,SMA not useful for parkinson)
    MotorCortex = nengo.Network(label = "Motor Cortex")
    # Neural input node for M1
    with MotorCortex:
        M1Node = nengo.Node(output=1, label = "M1 node")
   
# For now we will suppose that the M1 arrives directly at shoulder and we are
# lifting an arm vertically in front of us (eccentric bicept movement).
with model:
    # Human Body model
    HumanBody = nengo.Network(label = "Human Body")
    with HumanBody:
        # Neural input to muscle transoform
        MuscleNet = nengo.Network(label = "Muscles")
        # Force to joint moment transform
        MomentArms = nengo.Network(label = "Moment Arms")
        # Joint moment to position
        Skeleton = nengo.Network(label = "Skeleton")
        #------- Muscle net -------#
        with MuscleNet:
            # Activation function
            def activation(t):
                act = 1/tauA * e^(-t/tauA)
                return act
            # Deactivation function
            def deactivation(t):
                deact = 1/tauD * e^(-t/tauD)
                return deact
            
                
            excTimeExponential = nengo.Node(
                output = lambda t: (1/tauNE) * mt.exp(-t/tauNE), 
                label = "Exp. for excitation")
            excitationNode = nengo.Node(output = lambda t,y: y[0]*y[1],
                size_in = 2, size_out = 1,label = "Excitation Node")
            ActExponential = nengo.Node(
                output = lambda t: (1/tauA) * mt.exp(-t/tauA), 
                label = "Exp. for activation")
            DeactExponential = nengo.Node(
                output = lambda t: -(1/tauD) * mt.exp(-t/tauD), 
                label = "Exp. for deactivation")   
                
            activationNode = nengo.Node(
                output = lambda t,y: y[0] * y[1] if y[0] >= y[3]
                    else y[0] * y[2],
                size_in = 4, size_out = 1,label = "Activation Node")
            contractionNode = nengo.Node(
                output = lambda t,y: y * f2l_rel * f2v_rel,
                size_in = 1, size_out = 1,label = "Contraction Dynamics Node")
        with MomentArms:
            MomentComp = nengo.Node(
                output = lambda t,y: r*y,
                size_in = 1, size_out = 1,label = "Moments")
        with Skeleton:
            ThetaDDComp = nengo.Node(
                output = lambda t,y: (y-Mp)/I,
                size_in = 1, size_out = 1,label = "thetaDD")
            ThetaDComp = nengo.Node(
                output = lambda t,y: y[1] + y[0] * dt,
                size_in = 2, size_out = 1,label = "thetaD")
            # Recurrent connection for integration of thetaD
            nengo.Connection(ThetaDComp,ThetaDComp[1])
            ThetaComp = nengo.Node(
                output = lambda t,y: dt*0.5*y[0]**2 + dt*y[1] + y[2],
                size_in = 3, size_out = 1,label = "theta")
            nengo.Connection(ThetaComp,ThetaComp[2])
# Connections cortex to body movements
with model:
    # Two parts into the node
    nengo.Connection(M1Node,excitationNode[0])
    nengo.Connection(excTimeExponential,excitationNode[1])
    # Activation node connections
    nengo.Connection(excitationNode,activationNode[0])
    nengo.Connection(ActExponential,activationNode[1])
    nengo.Connection(DeactExponential,activationNode[2])
    nengo.Connection(activationNode,activationNode[3])
    # Contraction node connection
    nengo.Connection(activationNode,contractionNode)
    # Calculate moment
    nengo.Connection(contractionNode,MomentComp)
    # Calculate angular accel.
    nengo.Connection(MomentComp,ThetaDDComp)
    # Calculate angular velocity
    nengo.Connection(ThetaDDComp,ThetaDComp[0])
    # Calculate angular movement
    nengo.Connection(ThetaDDComp,ThetaComp[0])
    nengo.Connection(ThetaDComp,ThetaComp[1])
    # Output position
    OutPos = nengo.Node(
                output = lambda t,y: r*y,
                size_in = 1, size_out = 1,label = "Effector Position")
        
    nengo.Connection(ThetaComp,OutPos)
# Nuclei, thalamus and BG
with model:    
    # PN stays between MC and CC/IO (for now its simple encoder)
    # RN stays between MC and IO (for now its simple encoder)
    RedNucleus = nengo.Network(label = "Red nucleus")
    PontineNucleus = nengo.Network(label = "Pontine nucleus")
    
    # Thalamus interpretes motor output from DCN toward the MC
    Thalamus = nengo.Network(label = "Thalamus")
    
    
# Olivocerebellar system
with model:
    # This is the olivocerebellar system
    OlivoCerebellar = nengo.Network(label = "OlivoCerebellar")
    with OlivoCerebellar:
        # Cerebellar Cortex receives MF input from pontine (motor command) and 
        # sensory feedback from MF input from the human body and CF input (error) 
        # from the IO
        CerebellarCortex = nengo.Network(label = "Cerebellar Cortex")
    
        # Inferior Olive receives MF input from pontine (motor command) and
        # through the red nucleus. Besides it receives sensory feedback from the body.
        # So far computes difference, and oscillates at STOs.
        InferiorOlive = nengo.Network(label = "Inferior Olive")
    
        # Deep Nuclei receive CF from the IO, motor command (t) from the PontineNucleus
        # It is only output of cerebellum toward the Motor Cortex ( MC(t+1)_correction_in_signal)
        DeepNuclei = nengo.Network(label = "Deep Cereballar Nuclei")
    
    
